In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the dataset
train_data = pd.read_csv('sign_mnist_train.csv')
test_data = pd.read_csv('sign_mnist_test.csv')

# Extract labels and images
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Remove 'J' (label 9) and 'Z' (label 25)
valid_labels = [i for i in range(26) if i not in [9, 25]]
train_mask = np.isin(y_train, valid_labels)
test_mask = np.isin(y_test, valid_labels)

X_train = X_train[train_mask]
y_train = y_train[train_mask]
X_test = X_test[test_mask]
y_test = y_test[test_mask]

# Adjust labels to be in range 0-23 instead of 0-24
y_train = [i if i < 9 else i - 1 for i in y_train]
y_test = [i if i < 9 else i - 1 for i in y_test]

# Convert lists to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Reshape images to 32x32 (since the images consists of 32x32 pixels)
X_train = X_train.reshape(-1, 32, 32, 1)
X_test = X_test.reshape(-1, 32, 32, 1)

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Create a fixed validation set and a test set from the testing data
X_val, X_final_test, y_val, y_final_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# One-hot encode labels
y_train_enc = to_categorical(y_train, num_classes=24)
y_val_enc = to_categorical(y_val, num_classes=24)
y_final_test_enc = to_categorical(y_final_test, num_classes=24)


In [7]:
# Let's see how big it is
print(X_train.shape)
print(X_test.shape)
n_total = X_train.shape[0]

(27455, 32, 32, 1)
(7172, 32, 32, 1)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define the densely connected model
dense_model = Sequential([
    Flatten(input_shape=(32, 32, 1)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_dense = dense_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))


Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2802 - loss: 2.3809 - val_accuracy: 0.6043 - val_loss: 1.2454
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6936 - loss: 0.9229 - val_accuracy: 0.6511 - val_loss: 1.0503
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8416 - loss: 0.4874 - val_accuracy: 0.7078 - val_loss: 0.8804
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9111 - loss: 0.2799 - val_accuracy: 0.7426 - val_loss: 0.8748
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9519 - loss: 0.1584 - val_accuracy: 0.6997 - val_loss: 1.1686
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9662 - loss: 0.1124 - val_accuracy: 0.7772 - val_loss: 0.9599
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9553 - loss: 0.1395 - val_accuracy: 0.7998 - val_loss: 0.9480
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9742 - loss: 0.0812 - val_accuracy: 0.7599 - val_

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_cnn = cnn_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))


Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5699 - loss: 1.4749 - val_accuracy: 0.8667 - val_loss: 0.4074
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9978 - loss: 0.0216 - val_accuracy: 0.8846 - val_loss: 0.4054
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 0.8673 - val_loss: 0.5328
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9947 - loss: 0.0209 - val_accuracy: 0.8938 - val_loss: 0.4102
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 3.1992e-04 - val_accuracy: 0.8918 - val_loss: 0.4420
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.8230e-04 - val_accuracy: 0.8985 - val_loss: 0.4202
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.1571e-04 - val_accuracy: 0.8963 - val_loss: 0.4635
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 7.1300e-05 - val_accurac

In [11]:
def evaluate_model(model_name, X_final_test, y_final_test_enc, verbose = 2):
    model_eval = model_name.evaluate(X_final_test, y_final_test_enc, verbose = verbose)
    print(f"Dense Model - Test Accuracy: {model_eval[1]}")

    # Detailed evaluation of the model
    y_pred = np.argmax(model_name.predict(X_final_test), axis=1)

    # Compute accuracy per class, skipping index 9 (for J)
    accuracy_per_class = []
    for i in range(24):
        if np.sum(y_final_test == i) > 0:
            accuracy_per_class.append(np.mean(y_pred[y_final_test == i] == i))
        else:
            accuracy_per_class.append(np.nan)  # Handle classes with no samples

    # Filter out NaN values to calculate the median accuracy
    valid_accuracies = [acc for acc in accuracy_per_class if not np.isnan(acc)]
    median_accuracy = np.median(valid_accuracies)

    print(f"Unbiased Median Accuracy: {median_accuracy}")

    # Identify the letter with the highest individual accuracy
    highest_accuracy_class = np.nanargmax(accuracy_per_class)
    print(f"Letter with Highest Accuracy: {chr(highest_accuracy_class + ord('A'))}")

    # Identify the letter with the lowest individual accuracy
    lowest_accuracy_class = np.nanargmin(accuracy_per_class)
    print(f"Letter with Lowest Accuracy: {chr(lowest_accuracy_class + ord('A'))}")

    # Calculate the confusion matrix
    from sklearn.metrics import confusion_matrix
    conf_matrix = confusion_matrix(y_final_test, y_pred)

    # Set the diagonal elements to zero to exclude correct classifications
    np.fill_diagonal(conf_matrix, 0)

    # Find the indices of the top three errors
    errors = np.unravel_index(np.argsort(-conf_matrix, axis=None), conf_matrix.shape)

    # Get the top three most common errors
    common_errors = [(chr(errors[0][i] + ord('A')), chr(errors[1][i] + ord('A'))) for i in range(3)]
    print(f"Most Common Errors: {common_errors}")

    # Report overall mean accuracy and accuracy per letter
    mean_accuracy = np.nanmean(accuracy_per_class)
    print(f"Overall Mean Accuracy: {mean_accuracy}")

    # Print each letter and its accuracy
    letters = [chr(i + ord('A')) for i in range(26) if i not in [9, 25]]
    for i, acc in enumerate(accuracy_per_class):
        print(f"Letter {letters[i]}: Accuracy {acc}")




In [12]:
evaluate_model(dense_model, X_final_test, y_final_test_enc, 2)

113/113 - 0s - 836us/step - accuracy: 0.6938 - loss: 1.5337
Dense Model - Test Accuracy: 0.6938092708587646
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step
Unbiased Median Accuracy: 0.7361098285468033
Letter with Highest Accuracy: K
Letter with Lowest Accuracy: R
Most Common Errors: [('N', 'S'), ('G', 'H'), ('L', 'I')]
Overall Mean Accuracy: 0.6839723408333663
Letter A: Accuracy 0.9878048780487805
Letter B: Accuracy 0.751131221719457
Letter C: Accuracy 0.8486842105263158
Letter D: Accuracy 0.46511627906976744
Letter E: Accuracy 0.8690476190476191
Letter F: Accuracy 0.47692307692307695
Letter G: Accuracy 0.7625
Letter H: Accuracy 0.9248826291079812
Letter I: Accuracy 0.7210884353741497
Letter K: Accuracy 0.6481481481481481
Letter L: Accuracy 1.0
Letter M: Accuracy 0.45
Letter N: Accuracy 0.45390070921985815
Letter O: Accuracy 0.5041322314049587
Letter P: Accuracy 0.88
Letter Q: Accuracy 0.8133333333333334
Letter R: Accuracy 0.8450704225352113
Letter S: Accuracy 0.1864406779661017
Letter T: A

In [13]:
evaluate_model(cnn_model, X_final_test, y_final_test_enc, 2)

113/113 - 0s - 1ms/step - accuracy: 0.9186 - loss: 0.4880
Dense Model - Test Accuracy: 0.9185722470283508
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Unbiased Median Accuracy: 0.9267629211132036
Letter with Highest Accuracy: A
Letter with Lowest Accuracy: Q
Most Common Errors: [('L', 'R'), ('T', 'Q'), ('S', 'H')]
Overall Mean Accuracy: 0.9094666290279649
Letter A: Accuracy 1.0
Letter B: Accuracy 1.0
Letter C: Accuracy 0.9078947368421053
Letter D: Accuracy 1.0
Letter E: Accuracy 0.9563492063492064
Letter F: Accuracy 1.0
Letter G: Accuracy 0.95625
Letter H: Accuracy 0.9859154929577465
Letter I: Accuracy 0.9047619047619048
Letter K: Accuracy 0.9382716049382716
Letter L: Accuracy 1.0
Letter M: Accuracy 0.8
Letter N: Accuracy 0.8581560283687943
Letter O: Accuracy 0.8842975206611571
Letter P: Accuracy 1.0
Letter Q: Accuracy 1.0
Letter R: Accuracy 0.5211267605633803
Letter S: Accuracy 0.9152542372881356
Letter T: Accuracy 0.7286821705426356
Letter U: Accuracy 0.8382352941176471
Letter V: Accurac

Part 2

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=10,        # Randomly rotate images by 10 degrees
    width_shift_range=0.1,    # Randomly translate images horizontally by 10% of the width
    height_shift_range=0.1,   # Randomly translate images vertically by 10% of the height
    zoom_range=0.1,           # Randomly zoom images by 10%
    horizontal_flip=True      # Randomly flip images horizontally
)

# Fit the generator to the training data
datagen.fit(X_train)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

def create_dense_model_with_regularization(layer_sizes, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 1)))
    for size in layer_sizes:
        model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

def create_cnn_model_with_regularization(conv_layers, dense_layers, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    for filters, kernel_size in conv_layers:
        model.add(Conv2D(filters, kernel_size, activation='relu', padding='same', kernel_regularizer=l2(l2_lambda), input_shape=(32, 32, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Flatten())
    for size in dense_layers:
        model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [28]:
# Define different dense models to experiment with
dense_models = [
    create_dense_model_with_regularization([512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256, 128], learning_rate=0.001)
]

# Train and evaluate each dense model
dense_histories = []
for dense_model in dense_models:
    print(f'Dense model: {dense_model}')
    history = dense_model.fit(
        datagen.flow(X_train, y_train_enc, batch_size=30),
        steps_per_epoch=len(X_train) // 30,
        epochs=50,
        validation_data=(X_val, y_val_enc),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    val_accuracy = history.history['val_accuracy'][-1]
    dense_histories.append((val_accuracy, dense_model))

Dense model: <Sequential name=sequential_14, built=True>
Epoch 1/50


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 26/915 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0723 - loss: 13.5937    

c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


915/915 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2309 - loss: 7.0087 - val_accuracy: 0.1277 - val_loss: 5.1259 - learning_rate: 0.0010
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 137us/step - accuracy: 0.4333 - loss: 2.3050 - val_accuracy: 0.1244 - val_loss: 5.0759 - learning_rate: 0.0010
Epoch 3/50
  1/915 ━━━━━━━━━━━━━━━━━━━━ 35s 39ms/step - accuracy: 0.2333 - loss: 2.8634

c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3064 - loss: 2.6218 - val_accuracy: 0.1698 - val_loss: 5.0103 - learning_rate: 0.0010
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 136us/step - accuracy: 0.2667 - loss: 2.4667 - val_accuracy: 0.1743 - val_loss: 4.9178 - learning_rate: 0.0010
Epoch 5/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3849 - loss: 2.2518 - val_accuracy: 0.1194 - val_loss: 7.0874 - learning_rate: 0.0010
Dense model: <Sequential name=sequential_15, built=True>
Epoch 1/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.2217 - loss: 10.8565 - val_accuracy: 0.0418 - val_loss: 7.7017 - learning_rate: 0.0010
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step - accuracy: 0.2000 - loss: 3.3610 - val_accuracy: 0.0569 - val_loss: 7.1467 - learning_rate: 0.0010
Epoch 3/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.3034 - loss: 2.8370 - val_accuracy: 0.0477 - val_loss: 7.2845 - learning_rate: 0.0010
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━

In [23]:
# Define different CNN models with regularization to experiment with
cnn_models = [
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3))], [128],learning_rate=0.001),
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3)), (128, (3, 3))], [256], learning_rate=0.001),
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3)), (128, (3, 3)), (256, (3, 3))], [512], learning_rate=0.001)
]

# Train and evaluate each CNN model
cnn_histories = []
for cnn_model in cnn_models:
    print(f'CNN model: {cnn_model}')
    history = cnn_model.fit(
        datagen.flow(X_train, y_train_enc, batch_size=30),
        steps_per_epoch=len(X_train) // 30,
        epochs=50,
        validation_data=(X_val, y_val_enc),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    val_accuracy = history.history['val_accuracy'][-1]
    cnn_histories.append((val_accuracy, cnn_model))


CNN model: <Sequential name=sequential_11, built=True>
Epoch 1/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5056 - loss: 3.5544 - val_accuracy: 0.7091 - val_loss: 1.6448 - learning_rate: 0.0010
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step - accuracy: 0.8667 - loss: 1.1777 - val_accuracy: 0.7119 - val_loss: 1.5918 - learning_rate: 0.0010
Epoch 3/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8417 - loss: 1.2361 - val_accuracy: 0.3960 - val_loss: 4.5198 - learning_rate: 0.0010
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.8333 - loss: 1.0824 - val_accuracy: 0.4030 - val_loss: 4.5538 - learning_rate: 0.0010
Epoch 5/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8731 - loss: 1.0464 - val_accuracy: 0.7719 - val_loss: 1.2549 - learning_rate: 0.0010
CNN model: <Sequential name=sequential_12, built=True>
Epoch 1/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5937 - loss: 4.8820 - val_accuracy: 0.7931 - val_loss: 1.48

In [24]:
# Combine all histories and models
all_histories = dense_histories + cnn_histories

# Select the best model based on validation accuracy
best_model = max(all_histories, key=lambda x: x[0])[1]

# Evaluate the best model on the final test set
final_eval = best_model.evaluate(X_final_test, y_final_test_enc, verbose=0)
print(f"Best Model - Final Test Accuracy: {final_eval[1]}")


Best Model - Final Test Accuracy: 0.9994422793388367


In [27]:
from sklearn.metrics import confusion_matrix

# Predict on the final test set
y_pred = np.argmax(best_model.predict(X_final_test), axis=1)

# Compute accuracy per class
accuracy_per_class = []
for i in range(24):
    if np.sum(y_final_test == i) > 0:
        accuracy = np.mean(y_pred[y_final_test == i] == i)
        accuracy_per_class.append(accuracy)
    else:
        accuracy_per_class.append(np.nan)  # Handle classes with no samples

# Filter out NaN values to calculate the median accuracy
valid_accuracies = [acc for acc in accuracy_per_class if not np.isnan(acc)]
median_accuracy = np.median(valid_accuracies)

print(f"Unbiased Median Accuracy: {median_accuracy}")

# Identify the letter with the highest individual accuracy
highest_accuracy_class = np.nanargmax(accuracy_per_class)
print(f"Letter with Highest Accuracy: {chr(highest_accuracy_class + ord('A'))}")

# Identify the letter with the lowest individual accuracy
lowest_accuracy_class = np.nanargmin(accuracy_per_class)
print(f"Letter with Lowest Accuracy: {chr(lowest_accuracy_class + ord('A'))}")

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_final_test, y_pred)

# Set the diagonal elements to zero to exclude correct classifications
np.fill_diagonal(conf_matrix, 0)

# Find the indices of the top three errors
errors = np.unravel_index(np.argsort(-conf_matrix, axis=None), conf_matrix.shape)

# Get the top three most common errors
common_errors = [(chr(errors[0][i] + ord('A')), chr(errors[1][i] + ord('A'))) for i in range(3)]
print(f"Most Common Errors: {common_errors}")


# Report overall mean accuracy and accuracy per letter
mean_accuracy = np.nanmean(accuracy_per_class)
print(f"Overall Mean Accuracy: {mean_accuracy}")

# Print each letter and its accuracy
letters = [chr(i + ord('A')) for i in range(26) if i not in [9, 25]]
for i, acc in enumerate(accuracy_per_class):
    print(f"Letter {letters[i]}: Accuracy {acc}")


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Unbiased Median Accuracy: 1.0
Letter with Highest Accuracy: A
Letter with Lowest Accuracy: B
Most Common Errors: [('E', 'L'), ('B', 'T'), ('P', 'V')]
Overall Mean Accuracy: 0.9996461191314133
Letter A: Accuracy 1.0
Letter B: Accuracy 0.995475113122172
Letter C: Accuracy 1.0
Letter D: Accuracy 1.0
Letter E: Accuracy 0.996031746031746
Letter F: Accuracy 1.0
Letter G: Accuracy 1.0
Letter H: Accuracy 1.0
Letter I: Accuracy 1.0
Letter K: Accuracy 1.0
Letter L: Accuracy 1.0
Letter M: Accuracy 1.0
Letter N: Accuracy 1.0
Letter O: Accuracy 1.0
Letter P: Accuracy 1.0
Letter Q: Accuracy 1.0
Letter R: Accuracy 1.0
Letter S: Accuracy 1.0
Letter T: Accuracy 1.0
Letter U: Accuracy 1.0
Letter V: Accuracy 1.0
Letter W: Accuracy 1.0
Letter X: Accuracy 1.0
Letter Y: Accuracy 1.0
